In [1]:
# import keras for model creation
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, GRU
from keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
# sklearn for labeling data
from sklearn.preprocessing import LabelEncoder
# numpy for loading data
import numpy as np

In [2]:
# load in computed training data
X_train_data = np.load("data_computed/X_train.npy")
Y_train_data = np.load("data_computed/Y_train.npy")
# load in computed testing data
X_test_data = np.load("data_computed/X_test.npy")
Y_test_data = np.load("data_computed/Y_test.npy")

In [3]:
# reshape data for CNN
X_train_data = X_train_data.reshape(X_train_data.shape[0],X_train_data.shape[1],X_train_data.shape[2],1)
X_test_data = X_test_data.reshape(X_test_data.shape[0],X_test_data.shape[1],X_test_data.shape[2],1)

In [4]:
# encode labels
encoder = LabelEncoder()
encoder.fit(Y_train_data)
Y_train_data = encoder.transform(Y_train_data).reshape([len(Y_train_data), 1])
encoder = LabelEncoder()
encoder.fit(Y_test_data)
Y_test_data = encoder.transform(Y_test_data).reshape([len(Y_test_data), 1])

## Create the model

In [5]:
# IMPORTANT: this code was partially taken from a blog online:
# https://medium.com/analytics-vidhya/music-classification-using-deep-learning-python-b22614adb7a2

# Initiating the model as Sequential
model = Sequential()

# Adding the CNN layers along with some drop outs and maxpooling
model.add(Conv2D(64, 2, activation = 'relu', input_shape = (X_train_data.shape[1:])))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.1))
model.add(Conv2D(128, 2, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.1))
model.add(Conv2D(256, 2, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (4,4)))
model.add(Dropout(0.1))
model.add(Conv2D(512, 2, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (6,6)))
model.add(Dropout(0.1))

# flattening the data to be passed to a dense layer
model.add(Flatten())

# Adding the dense layers
model.add(Dense(2048, activation = 'relu'))
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))

# final output layer with 10 predictions to be made
model.add(Dense(10, activation = 'softmax'))

# compile the model
# using adam as optimizer for learning rate
# using sparse categorical crossentropy as loss
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


## Training

In [14]:
# initialise saving model to disk
num_epochs = 50
checkpoint_filepath = 'models/model.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [15]:
# start training
history = model.fit(X_train_data, Y_train_data, validation_data=(X_test_data, Y_test_data),
                    epochs=num_epochs, callbacks=[model_checkpoint_callback])

Epoch 1/10
7/7 [==============================] - 54s 8s/step - loss: 2.3030 - accuracy: 0.1100 - val_loss: 2.3019 - val_accuracy: 0.1150
Epoch 2/10
7/7 [==============================] - 59s 9s/step - loss: 2.3015 - accuracy: 0.1100 - val_loss: 2.3005 - val_accuracy: 0.1450
Epoch 3/10
7/7 [==============================] - 59s 9s/step - loss: 2.3003 - accuracy: 0.1450 - val_loss: 2.2988 - val_accuracy: 0.1450
Epoch 4/10
7/7 [==============================] - 60s 9s/step - loss: 2.2983 - accuracy: 0.1450 - val_loss: 2.2970 - val_accuracy: 0.1450
Epoch 5/10
7/7 [==============================] - 61s 9s/step - loss: 2.2964 - accuracy: 0.1450 - val_loss: 2.2943 - val_accuracy: 0.1450
Epoch 6/10
7/7 [==============================] - 61s 9s/step - loss: 2.2935 - accuracy: 0.1450 - val_loss: 2.2912 - val_accuracy: 0.1450
Epoch 7/10
7/7 [==============================] - 61s 9s/step - loss: 2.2902 - accuracy: 0.1450 - val_loss: 2.2875 - val_accuracy: 0.1450
Epoch 8/10
7/7 [==================